# <b>A3.2 Redes Neuronales</b>

#### 1. Modelo de redes neuronales para la clasificación de de los dígitos 0 y 9.

#### 2. Evaluación la exactitud del modelo en conjunto de prueba y comparación con la exactitud de validación del modelo.

#### 3. Generación de 50 imágenes, 5 para cada dígito. Preprocesamiento para que puedan ser evaluadas de manera similar.

#### 4. Evaluación del desempeño de las imágenes. Comparación de los resultados con los previos y motivos de potencial error.

#### 5. 3 mejoras al sistema actual.

#### 6. Sistema de predicción en tiempo real los dígitos 0 al 9.

#### <b>Código de honor:</b> Doy mi palabra de que he realizado esta actividad con integridad académica.